# 1. Imagenes Oficiales

Las imagenes se almacenan en https://hub.docker.com/

Ejemplo imagen con mongo

- https://hub.docker.com/_/mongo

docker pull mongo

![](images/tag_image.png)

<b>Tag:</b> es una etiqueta que nos indica la versión de la imagen

### Descargar imagenes oficiales

docker pull <"NAME"> <"TAG">

Ejemplo:

- docker pull mongo
- docker pull mysql


### Listar imagenes

- docker images
- docker images | grep mysql #versiones

# 2. Crear imagenes oficiales

1. Creamos el Dockerfile

2. creamos las capas

    FROM centos

    RUN yum install httpd -y # "-y" afirmativo para todo en el proceso de instalación (instala apache)
3. contruimos la imagen con el comando
    - docker build --tag <"IMAGE-NAME">:<"TAG-NAME"> .  # Con el . indicamos que el dockerfile se encuentra en el directorio

![](images/docker_build.png)

# 3. Crear contenedor con base en la imagen

Para poder listar contenedores

- docker ps
- docker ps -a # Lista los activos


### Nota:
Para que los contenedores no mueran al contruirlos se debe implementar la capa <b>CMD</b> en el docker file, ejemplo:
    - CMD apachectl -DFOREGROUND  #correr el servicio de apache en primer plano
    
Luego de agregarle la capa debemos volver a construir la imagen y luego ya podremos contruir el contenedor

### RUN container

  -<b>Command</b>: docker run -d --name <"CONTAINER-NAME"> <"IMAGE-NAME">:<"TAG-NAME">

Ejemplo:

    - docker run -d --name apache apache-centos:apache-cmd

![](images/docker_run.png)

Contenedores activos

![](images/docker_ps.png)

### Correr por un purto especifico

- docker run -d -p 80:80 <"CONTAINER_NAME"> <"IMAGE_NAME">:<"TAG">
- Ejemplo: docker run -d -p 80:80 apache1 apache-centos:apache-cmd

### Eliminar containers

    - docker rm -fv <"CONTAINER_NAME">

# 4. DOCKER FILE

![](images/docker_File.png)

### COPY 
copiar archivos a la imagen  
ADD: tambien cumple la función de copy pero con la ventaja que podemos usar urls y este comando descarga el contenido de la web.

Ejemplos:

- FROM centos

- RUN yum install httpd -y

- COPY frontend /var/www/html

- CMD apachectl -DFOREGROUND

### ADD

- FROM centos

- RUN yum install httpd -y

- ADD frontend /var/www/html

- CMD apachectl -DFOREGROUND

### ENV

Sirve para agregar variables de entornos

- ENV <"name"> <"content">, ejemplo: ENV contenido prueba

Para usar las variables de entorno usamos $, ejemplo: 


- RUN echo "$contenido" > /var/www/html/prueba.html, estamos creando un html con el contenido de la variable de entorno llamada contenido.


### WORKDIR

Sirve para definir donde vamos a trabajar dentro de la imagen, ejemplo:
   
- FROM centos

- RUN yum install httpd -y

- WORKDIR /var/www/html

- COPY frontend .

### EXPOSE

sirve para exponer puertos

### LABEL
Sirve para dar metadata a la imagen, usualmente se colocan iniciando el docker file(luego de FROM).

- LABEL version=1.0 
- LABEL description="This is an apache image"

### USER
Definir el usuario con el cual se ejecutan los comandos de la imagen, ejemplo:
Nota: El usuario debe existir antes de ser utilizado

- RUN useradd josearangos
- USER josearangos
- ... # Todas los comandos que se ejecuten de aquí en adelante seran con el usuario josearangos.


Ejemplo completo: 

FROM centos

RUN yum install httpd -y

WORKDIR /var/www/html

COPY frontend .

RUN echo "$(whoami)" > /var/www/html/user1.html

RUN useradd josearangos

USER josearangos

RUN echo "$(whoami)" > /tmp/user2.html

USER root

RUN cp /tmp/user2.html /var/www/html/user2.html

CMD apachectl -DFOREGROUND


### VOLUME
Forma de colocar la data persistente dentro del contenedor, para cuando este se elime se conserve.
Ejemplo:

- VOLUMEN /var/www/html


### CMD
Comando que mantiene vivo el contenedor, puede ser un script.

COPY run.sh /run.sh

CMD sh /run.sh


### Git Ignore 
Es usado para ignorar archivos o carpetas al momento de construir la images.
<b>Nota:</b> al momento de construir la imagen se recomienda tener solo los archivos necesario en el folder que contiene el dockerfile.

- .dockerignore
- se coloca el nombre de los archivos

# Logs de un contenedor.

docker logs -f <"CONTAINER_NAME">

## Ejemplo con todo 


FROM nginx

RUN useradd josearangos

COPY frontend /usr/share/nginx/html

ENV archivo docker

WORKDIR /usr/share/nginx/html

RUN echo "$archivo" > /usr/share/nginx/html/env.html

EXPOSE 90

LABEL version=2.3

USER josearangos

RUN echo "I'm $(whoami)" > /tmp/iam.html

USER root

RUN cp /tmp/iam.html /usr/share/nginx/html/docker.html

VOLUME /var/log/nginx

CMD nginx -g 'daemon off;'

# Buenas Prácticas

1. Efimero: Se debe permitir destruir con facilidad. 
2. Un solo servicio por contenedor (No se recomiendo un contenedor con dos imagenes(Apache/MySQL juntos ejemplo NO recomendado))
3. Build context ->.dockerignore Solo usar lo necesario (agregar archivos pesados)
4. Usar pocas capas
5. Varios argumentos en un sola capa
6. No instalar paquetes innecesarios.
7. Usar Labels

![]()

Ejemplo: Usar la misma capa

FROM nginx

RUN \ 
    echo "1" >> /usr/share/nginx/html/test.txt && \
    echo "2" >> /usr/share/nginx/html/test.txt && \
    echo "3" >> /usr/share/nginx/html/test.txt

Se uso el "&&" para que ejecutar en 1 sola capa 3 comandos  y "\" para mejorar la visualización de las instrucciones en el Docker file.

Mejora usando ENV

FROM nginx

ENV dir /usr/share/nginx/html/test.txt

RUN \
    
    - echo "1" >> $dir && \
    
    - echo "2" >> $dir && \
    
    - echo "3" >> $dir && \

![](images/buenas_practicas.png)

# Eliminar una Imagen

- docker rmi <"IMAGE_NAME>:<"TAG">

### Cambiar el nombre del dockerfile

Supongamos que el archivo se llama my-dockerfile, para construir la imagen apartir del docker file usamos el comando:

- docker build -t <"IMAGE-NAME"> -f <"FILE_NAME">

# Dangling Images (Imagenes huerfana) <"none"> 

Se soluciona usando los tags, cada vez que realice una modificación en el dockerfile debe existir una nueva imagen o tag.

#### Listar Imagenes Dangling
    - docker images -f dangling=true
### Eliminar Imagenes Dangling
    - docker images -f dangling=true -q | xargs docker rmi

# 5. Multi-Stage-Build

 Permite usar varios FROM
 